In [18]:
class BookNotFoundException(Exception):
    pass

class BookAlreadyBorrowedException(Exception):
    pass

class MemberLimitExceededException(Exception):
    pass

In [21]:
class Member:
    def __init__(self, name):
        self.name = name
        self.borrowed_books = []

    def can_borrow(self):
        return len(self.borrowed_books) < 3

    def borrow_book(self, book):
        if not self.can_borrow():
            raise MemberLimitExceededException(f"{self.name} has already borrowed the maximum of 3 books.")
        self.borrowed_books.append(book)

    def return_book(self, book):
        if book in self.borrowed_books:
            self.borrowed_books.remove(book)
        else:
            raise BookNotFoundException(f"{self.name} didn't borrow '{book.title}'")

    def __str__(self):
        return f"Member: {self.name}, Borrowed books: {len(self.borrowed_books)}"

In [20]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.is_borrowed = False

    def __str__(self):
        return f"'{self.title}' by {self.author} - {'Borrowed' if self.is_borrowed else 'Available'}"

In [24]:
class Library:
    def __init__(self):
        self.books = []
        self.members = []

    def add_book(self, title, author):
        self.books.append(Book(title, author))

    def add_member(self, name):
        self.members.append(Member(name))

    def find_book(self, title):
        for book in self.books:
            if book.title.lower() == title.lower():
                return book
        return None

    def find_member(self, name):
        for member in self.members:
            if member.name.lower() == name.lower():
                return member
        return None

    def borrow(self, member_name, book_title):
        member = self.find_member(member_name)
        book = self.find_book(book_title)

        if not book:
            raise BookNotFoundException(f"Book '{book_title}' not available")
        # Changed 'book.borrowed' to 'book.is_borrowed'
        if book.is_borrowed:
            raise BookAlreadyBorrowedException(f"Book '{book_title}' is already borrowed")
        if not member:
            raise Exception("Member not found")
        # The Member class uses borrowed_books and can_borrow, which is correct.
        if not member.can_borrow():
            raise MemberLimitExceededException(f"{member_name} has reached borrowing limit")

        # Changed 'book.borrowed' to 'book.is_borrowed'
        book.is_borrowed = True
        # Changed 'member.books.append(book)' to 'member.borrowed_books.append(book)'
        # to match the Member class's attribute name
        member.borrowed_books.append(book)
        print(f"{member_name} borrowed '{book_title}'")

    def return_book(self, member_name, book_title):
        member = self.find_member(member_name)
        book = self.find_book(book_title)

        if not book:
            raise BookNotFoundException(f"Book '{book_title}' not found")
        if not member:
            raise Exception("Member not found")
        # Changed 'member.books' to 'member.borrowed_books'
        if book not in member.borrowed_books:
            raise Exception(f"{member_name} didn't borrow this book")

        book.is_borrowed = False
        # Changed 'member.books.remove(book)' to 'member.borrowed_books.remove(book)'
        member.borrowed_books.remove(book)
        print(f"{member_name} returned '{book_title}'")

    def show_books(self):
        print("\nLibrary Books:")
        for book in self.books:
            # Changed 'book.borrowed' to 'book.is_borrowed'
            status = "Borrowed" if book.is_borrowed else "Available"
            print(f"'{book.title}' by {book.author} - {status}")

    def show_members(self):
        print("\nLibrary Members:")
        for member in self.members:
            # Changed 'len(member.books)' to 'len(member.borrowed_books)'
            print(f"{member.name} ({len(member.borrowed_books)} books borrowed)")

In [25]:
def test_library():
    library = Library()

    # Adding
    library.add_book("Atomic Habits", "Author")
    library.add_book("Harry Potter", "Ms")
    library.add_book("Pride and Prejudice", "Jane Austen")

    # Add members
    library.add_member("Me")
    library.add_member("Her")

    # Showing the status
    library.show_books()
    library.show_members()

    # Test borrowing
    try:
        library.borrow("Me", "Atomic Habits")
        library.borrow("Me", "Harry Potter")
        library.borrow("Me", "Pride and Prejudice")
        library.borrow("Me", "Harry Potter")  # already borrowed
    except Exception as e:
        print(f"\nError: {e}")

    try:
        library.borrow("Her", "Non-existent Book")  # book not found
    except Exception as e:
        print(f"\nError: {e}")

    # Test returning
    try:
        library.return_book("Her", "Pride and Prejudice")
        library.borrow("Her", "Atomic Habits")
    except Exception as e:
        print(f"\nError: {e}")


    library.show_books()
    library.show_members()

test_library()


Library Books:
'Atomic Habits' by Author - Available
'Harry Potter' by Ms - Available
'Pride and Prejudice' by Jane Austen - Available

Library Members:
Me (0 books borrowed)
Her (0 books borrowed)
Me borrowed 'Atomic Habits'
Me borrowed 'Harry Potter'
Me borrowed 'Pride and Prejudice'

Error: Book 'Harry Potter' is already borrowed

Error: Book 'Non-existent Book' not available

Error: Her didn't borrow this book

Library Books:
'Atomic Habits' by Author - Borrowed
'Harry Potter' by Ms - Borrowed
'Pride and Prejudice' by Jane Austen - Borrowed

Library Members:
Me (3 books borrowed)
Her (0 books borrowed)
